<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/low_level/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Building Evaluation from Scratch

We show how you can build evaluation modules from scratch. This includes both evaluation of the final generated response (where the output is plain text), as well as the evaluation of retrievers (where the output is a ranked list of items).

We have in-house modules in our [Evaluation](https://gpt-index.readthedocs.io/en/latest/core_modules/supporting_modules/evaluation/root.html) section.

## Setup

We load some data and define a very simple RAG query engine that we'll evaluate (uses top-k retrieval).

In [1]:
# %pip install llama-index-readers-file pymupdf
# %pip install llama-index-llms-openai

In [1]:
import logging
import sys
import pandas as pd
import numpy as np
import json
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

In [2]:
from pathlib import Path
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter, SimpleNodeParser
from llama_index.llms.openai import OpenAI

In [3]:
# load yaml file
import yaml
from easydict import EasyDict
config = EasyDict(yaml.safe_load(open("defaults.yaml")))
config

{'QUESTION_GEN_SYS_TMPL': 'You are a law Professor.\nYour task is to setup {num_questions_per_chunk} questions for an upcoming quiz.\nThe questions should be diverse in nature across the document.\nRestrict the questions to the context information provided.\nThe questions should be in Arabic.\n',
 'QUESTION_GEN_SYS_TMPL_V2': 'You are a law Professor.\nYour task is to setup {num_questions_per_chunk} questions for an upcoming quiz.\nThe questions should be diverse in nature across the document.\nRestrict the questions to the context information provided.\nA student should be able to answer your questions using only the context information.\nAvoid asking about dates or specific numbers.\nAsk short questions only\nQuesitons should be solvable with short answers\nThe questions should be in Arabic.\n',
 'QUESTION_GEN_USER_TMPL': 'Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, generate the rele

In [4]:
from datasets import load_dataset
import os

if os.path.exists('sources/ArabicMMLU.csv'):
    arabicmmlu_df = pd.read_csv('sources/ArabicMMLU.csv')
else:
    arabicMMLU = load_dataset('MBZUAI/ArabicMMLU')
    arabicmmlu_df = arabicMMLU['test'].to_pandas()
    arabicmmlu_df.to_csv('source/ArabicMMLU.csv', index=False)
    

arabicmmlu_df

,ID,Source,Country,Group,Subject,Level,Question,Context,Answer Key,Option 1,Option 2,Option 3,Option 4,Option 5,is_few_shot
0,0,https://folderat.com/Reference/1649/%D8%A3%D8%...,NaN,Humanities,Islamic Studies,NaN,كم عدد سور القرآن الكريم؟,NaN,D,111,112,113,114,NaN,0
1,1,https://folderat.com/Reference/1649/%D8%A3%D8%...,NaN,Humanities,Islamic Studies,NaN,في أي الأيام خلق سيدنا آدم عليه السلام؟,NaN,D,يوم السبت,يوم الاثنين,يوم الأربعاء,يوم الجمعة,NaN,0
2,2,https://folderat.com/Reference/1649/%D8%A3%D8%...,NaN,Humanities,Islamic Studies,NaN,كم عدد السنوات التي نام فيها أهل الكهف؟,NaN,B,300 سنة,309 سنوات,400 سنة,409 سنوات,NaN,0
3,3,https://folderat.com/Reference/1649/%D8%A3%D8%...,NaN,Humanities,Islamic Studies,NaN,اسم أول صحابي قرأ القرآن جهرة؟,NaN,A,عبد الله بن مسعود,عبد الله بن عباس,عبد الله بن عمرو بن العاص,عبد الله بن الزبير,NaN,0
4,4,https://folderat.com/Reference/1649/%D8%A3%D8%...,NaN,Humanities,Islamic Studies,NaN,من هي أخر زوجات النبي التي توفيت؟,NaN,C,السيدة زينب بنت جحش,السيدة عائشة بنت أبي بكر,السيدة أم سلمة,السيدة سودة بنت زمعة,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14570,14570,exams-28a8e94f-7722-11ea-9116-54bef70b159e,NaN,Social Science,Civics,High,شهدت دولة قطر نقلة نوعية في حرية الرأي وذلك من...,NaN,B,إنشاء الهيئة العامة للإذاعة والتلفزيون,الانضمام لمجلس التعاون الخليجي,قرار إلغاء وزارة الإعلام,انتخابات المجلس البلدي المركزي,NaN,0
14571,14571,exams-28a8e950-7722-11ea-9116-54bef70b159e,NaN,Social Science,Civics,High,من المبادرات الوطنية القطرية للصحة:,NaN,B,برنامج الطالب النظيف,برنامج دلني على فطرتي,موقع الأمن والسلامة,الجمعية الوطنية للسكري,NaN,0
14572,14572,exams-28a8e994-7722-11ea-9116-54bef70b159e,NaN,Social Science,Civics,High,تهدف التنمية المستدامة في العالم الإسلامي في ا...,NaN,B,توظيف الموارد من أجل رفع المستوى المعيشي للسكان.,تحقيق الاستخدام الأمثل للأراضي الزراعية.,تطوير وتنمية سكان العالم الإسلامي.,رفع مستوى الخدمات الصحية كافة المجالات .,NaN,0
14573,14573,exams-28a8e9a4-7722-11ea-9116-54bef70b159e,NaN,Social Science,Civics,High,تهدف التنمية المستدامة في العالم الإسلامي في ا...,NaN,C,توظيف الموارد من أجل رفع المستوى المعيشي للسكان .,رفع مستوى الخدمات الصحية كافة المجالات .,تحقيق الاستخدام الأمثل للأراضي الزراعية.,تطوير وتنمية سكان العالم الإسلامي.,NaN,0


In [ ]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1e6)



def convert_qa_to_string(row) -> str:
    options = list(map(str.strip, filter(lambda x: x not in [None, np.nan, 'nan'], [
       str(row[f"Option {i}"]) for i in range(1, 6)
    ])))
    options_str = "\n".join(
        f"{i+1}. {x}" for i, x in enumerate(options)
    )
    answer_key_map = {
        "A": 0,
        "B": 1,
        "C": 2,
        "D": 3,
        "E": 4,
    }
    correct_answer_str = options[answer_key_map[row["Answer Key"]]]

    return config.MCQ_PROMPT.format(
        question=row['Question'],
        options=options_str,
        correct_answer=correct_answer_str
    )


documents = [
    Document(
        text=convert_qa_to_string(row),
        metadata=row.to_dict()
    ) for _, row in arabicmmlu_df.query('Subject=="Law"').iterrows()
]

arabicmmlu_nodes = node_parser.get_nodes_from_documents(
    documents,
    show_progress=True
)

In [6]:
moj_df = pd.read_csv('sources/MOJ_Regulations.csv')
print(moj_df.shape)
moj_df.head()

(5751, 5)


,Parent,Chapter,Section,Description,Tables
0,المادة الأولى,NaN,NaN,يقصد بالألفاظ والعبارات الآتية – أينما وردت في...,[]
1,المادة الثانية,NaN,NaN,١- تقدم مذكرة الاعتراض إلى محكمة الدرجة الأولى...,[]
2,المادة الثالثة,NaN,NaN,١- للدائرة مصدرة الحكم الاطلاع على مذكرة الاعت...,[]
3,المادة الرابعة,NaN,NaN,الغائب المحكوم عليه الوارد في الفقرة (٤) من ال...,[]
4,المادة الخامسة,NaN,NaN,١- طلب الاستئناف ينقل الدعوى بحالتها التي كانت...,[]


In [7]:
tameem_df = pd.read_csv('sources/tameem.csv')
print(tameem_df.shape)
tameem_df.head()

(338, 4)


,رقم التعميم,موضوعه,تاريخه,نص التعميم
0,109/1/ك,حوادث السيارات,1392/7/3,وبعد: (فنشير إلى خطاب صاحب السمو الملكي نائب و...
1,109/1/ك,الديـات,1392/7/3,وبعد: (فنشير إلى خطاب صاحب السمو الملكي نائب و...
2,109/1/ك,الطب,1392/7/3,- وبعد: (فنشير إلى خطاب صاحب السمو الملكي نائب...
3,109/1/ك,قتل الخطأ,1392/7/3,- وبعد: (فنشير إلى خطاب صاحب السمو الملكي نائب...
4,114/3/ت,حوادث السيارات,1398/6/21,وبعد: (نشير إلى خطاب سماحة الرئيس العام لإدارا...


In [8]:
moj_docs = [ 
            Document(
                text=row['Description'],
                metadata=row.drop('Description').to_dict()
            ) for _, row in moj_df.iterrows()
]

In [9]:
tameem_docs = [
    Document(
        text=row['نص التعميم'],
        metadata=row.drop('نص التعميم').to_dict()
    ) for _, row in tameem_df.iterrows()
]

In [10]:
source_docs = moj_docs + tameem_docs
len(source_docs)

6089

In [31]:
# llama index format
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.llms.anthropic import Anthropic
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
CLAUDE_API_KEY = os.getenv("CLAUDE_API_KEY")
api_version = "2024-02-15-preview"

llm_gpt4 = AzureOpenAI(
    engine="gpt-4",
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=api_version,
)

llm_claude3 = Anthropic(
    'claude-3-opus-20240229',
    api_key=CLAUDE_API_KEY
)

In [12]:
node_parser = SentenceSplitter(chunk_size=1024)

In [13]:
from llama_index.embeddings.openai import OpenAIEmbedding

OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")

from llama_index.core import Settings
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import PromptTemplate, ServiceContext, StorageContext, VectorStoreIndex, load_index_from_storage

embed_model = OpenAIEmbedding(model='text-embedding-3-large', api_key=OPENAI_API_KEY)

Settings.embed_model = embed_model

In [ ]:
Settings.llm = llm_claude3

chroma_client = chromadb.PersistentClient(path='./chroma_db')
# Traditional VDB
try:
    chroma_collection = chroma_client.get_collection(f'ArabicMMLU_legal')
except Exception as e:
    print("Creating new collection")
    chroma_collection = chroma_client.create_collection('ArabicMMLU_legal')

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

arabicmmlu_index = VectorStoreIndex(
    arabicmmlu_nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    use_async=False,
    show_progress=True,
)
# arabicmmlu_retriever = arabicmmlu_index.as_retriever(
#     similarity_top_k=3,
#     embed_model=embed_model,
# )
# # Sentence window retrieval
# query_engine_sentence_window = index_sentence_window.as_query_engine(
#     text_qa_template=text_qa_template, similarity_top_k=3, embed_model=embed_model, llm=llm
# )

In [ ]:
node_parser = SentenceSplitter(chunk_size=1e6, chunk_overlap=0)

nodes = node_parser.get_nodes_from_documents(
    source_docs,
    show_progress=True
)

## Dataset Generation

We first go through an exercise of generating a synthetic evaluation dataset. We do this by synthetically generating a set of questions from existing context. We then run each question with existing context through a powerful LLM (e.g. GPT-4) to generate a "ground-truth" response.

### Define Functions

We define the functions that we will use for dataset generation:

In [16]:
from llama_index.core.schema import BaseNode
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate, PromptTemplate
from typing import Tuple, List
import re
from tqdm.auto import tqdm
from multiprocessing.pool import ThreadPool

We define `generate_answers_for_questions` to generate answers from questions given context.

In [102]:
question_answer_template = ChatPromptTemplate(
    message_templates=[
        ChatMessage(role=MessageRole.USER, content=config.QA_PROMPT),
    ]
)

def generate_answers_for_questions(
    questions: List[str], context: str, llm: OpenAI
) -> str:
    """Generate answers for questions given context."""
    
    def generate_answer(idx, question):
        fmt_qa_prompt = question_answer_template.format_messages(
            context_str=context,
            query_str=question,
        )
        response_obj = llm.chat(fmt_qa_prompt)
        return response_obj.message.content

    # for idx, node in enumerate(nodes):
    answers = list(
        tqdm(
            ThreadPool().imap(
                lambda x: generate_answer(*x),
                enumerate(questions),
            ),
        "generate_answers_for_questions()",
        total=len(questions),
        )
    )

    return answers

We define `generate_qa_pairs` to generate qa pairs over an entire list of Nodes.

In [103]:
question_gen_template = ChatPromptTemplate(
    message_templates=[
        ChatMessage(role=MessageRole.SYSTEM, content=config.QUESTION_GEN_SYS_TMPL),
        ChatMessage(role=MessageRole.USER, content=config.QUESTION_GEN_USER_TMPL),
    ]
)


def generate_qa_pairs(
    nodes: List[BaseNode], llm: OpenAI, num_questions_per_chunk: int = 2,
    delimiter: str = "\n",
    question_gen_template=question_gen_template,
) -> List[Tuple[str, str]]:
    """Generate questions."""
    def process_node(idx, node):
        context_str = node.get_content(metadata_mode="all")
        fmt_messages = question_gen_template.format_messages(
            num_questions_per_chunk=num_questions_per_chunk,
            context_str=context_str,
        )
        chat_response = llm.chat(fmt_messages)
        raw_output = chat_response.message.content

        result_list = str(raw_output).strip().split(delimiter)
        cleaned_questions = [
            re.sub(r"^\d+[\).\s]", "", question).strip()
            for question in result_list
        ]
        answers = generate_answers_for_questions(
            cleaned_questions, context_str, llm
        )
        cur_qa_pairs = list(zip(cleaned_questions, answers))
        return cur_qa_pairs
    
    qa_pairs = list(
        tqdm(
            ThreadPool().imap(
                lambda x: process_node(*x),
                enumerate(nodes),
            ),
        "Generating QA pairs",
        total=len(nodes),
        )
    )
    # flatten
    qa_pairs = [item for sublist in qa_pairs for item in sublist]
        
    return qa_pairs


In [ ]:
qa_pairs = generate_qa_pairs(
    nodes,
    llm=llm_gpt4,
    num_questions_per_chunk=1,
)

Converting question answer paris int MSQs

In [109]:
qa_to_mcq_template = ChatPromptTemplate(
    message_templates=[
        ChatMessage(role=MessageRole.USER, content=config.QA_TO_MCQ_PROMPT),
    ]
)
qa_to_mcq_cot_template = ChatPromptTemplate(
    message_templates=[
        ChatMessage(role=MessageRole.USER, content=config.QA_TO_MCQ_COT_PROMPT),
    ]
)

In [111]:
def convert_quesitons_to_mcqs(
    qa_pairs: List[tuple], mcq_prompt_template: str, llm: OpenAI
) -> str:
    """Converting question-answer paris into MCQs."""
    
    def question_to_mcq(idx, qa_pair):
        question, answer = qa_pair
        prompt_template = mcq_prompt_template.format_messages(
            question=question,
            answer=answer,
        )
        response_obj = llm.chat(prompt_template)
        return response_obj.message.content

    mcqs = list(
        tqdm(
            ThreadPool().imap(
                lambda x: question_to_mcq(*x),
                enumerate(qa_pairs),
            ),
        "convert_quesitons_to_mcqs()",
        total=len(qa_pairs),
        )
    )

    return mcqs

In [ ]:
mcqs = convert_quesitons_to_mcqs(
    qa_pairs,
    qa_to_mcq_template,
    llm_gpt4
)

In [ ]:
def format_mcqs(mcqs):
    formatted_mcqs = []
    for example in mcqs:
        question = example.split('\n')[0]
        options = example.split('\n')[2:]
        formatted_mcqs.append([question, options])
    return formatted_mcqs

formated_mcqs = format_mcqs(mcqs)
formated_mcqs

In [ ]:
qa_to_mcqs_dict = {}
for i in range(len(formated_mcqs)):
    qa_to_mcqs_dict[i] = {
        'question': formated_mcqs[i][0],
        'options': formated_mcqs[i][1],
        'answer': formated_mcqs[i][1][0],
        'context': nodes[i].text
    }

With Chain of Thought

In [ ]:
mcqs_cot = convert_quesitons_to_mcqs(
    qa_pairs,
    qa_to_mcq_cot_template,
    llm_gpt4
)

In [ ]:
def format_mcqs_cot(mcqs_cot):
    formated_mcqs_cot = []
    for example in mcqs_cot:
        example = example.strip('<start_thought>').split('<end_thought>')

        reasoning = example[0]
        question = example[1].strip().split('\n')[0]
        options = example[1].strip().split('\n')[2:]    
        
        answer = options[0]
        formated_mcqs_cot.append([question, reasoning, options, answer])
        
    return formated_mcqs_cot

formated_mcqs_cot = format_mcqs_cot(mcqs_cot)
formated_mcqs_cot

In [117]:
qa_to_mcqs_cot_dict = {}
for i in range(len(formated_mcqs_cot)):
    qa_to_mcqs_cot_dict[i] = {
        'question': formated_mcqs_cot[i][0],
        'reasoning': formated_mcqs_cot[i][1],
        'options': formated_mcqs_cot[i][2],
        'answer': formated_mcqs_cot[i][2][0],
        'context': nodes[i].text
    }

#### For MCQ Generation Using In-context Learning

In [36]:
import time

mcq_question_gen_template = ChatPromptTemplate(
    message_templates=[
        ChatMessage(role=MessageRole.SYSTEM, content=config.MCQ_QUESTION_GEN_SYS_TMPL),
        ChatMessage(role=MessageRole.USER, content=config.MCQ_QUESTION_GEN_USER_TMPL),
    ]
)


def generate_mcq_pairs(
    nodes_dict: dict,
    num_questions_per_chunk: int = 2,
    top_k: int = 3,
    delimiter: str = "####",
    mcq_question_gen_template=mcq_question_gen_template,
    
) -> List[Tuple[str, str]]:
    """Generate questions."""
    engine, nodes = list(nodes_dict.items())[0]
    if engine == 'gpt-4':
        llm = llm_gpt4
    else:
        llm = llm_claude3
    def process_node(idx, node):
        cur_context_str = node.get_content(metadata_mode="none")
        if "{few_shot_examples}" in '\n'.join([x.content for x in mcq_question_gen_template.message_templates]) and top_k > 0:
            arabicmmlu_retriever = arabicmmlu_index.as_retriever(
                similarity_top_k=top_k,
                embed_model=embed_model,
            )
            few_shot_examples_str = "\n\n".join([
                x.text for x in arabicmmlu_retriever.retrieve(node.text)
            ])
        else:
            few_shot_examples_str = ""

        fmt_messages = mcq_question_gen_template.format_messages(
            num_questions_per_chunk=num_questions_per_chunk,
            context_str=cur_context_str,
            few_shot_examples=few_shot_examples_str,
        )
        try:
            chat_response = llm.chat(fmt_messages)
            raw_output = chat_response.message.content
        except Exception as e:
            # add a 2 second sleep and retry
            time.sleep(2)
            try:
                chat_response = llm.chat(fmt_messages)
                raw_output = chat_response.message.content
            except Exception as e:
                raw_output = ''
        result_list = str(raw_output).strip().split(delimiter)
        # cur_mcq_pairs = [
        #     #TODO: make this from the config
        #     question.strip().strip('السؤال: ').split("الجواب الصحيح: ")
        #     for question in result_list if question.strip()
        # ]
        cur_mcq_pairs = [f'Engine: {engine}' + '\n\n' + f'Context: {node.text}\n\n' + x.strip() for x in result_list if x.strip()]
        
        return cur_mcq_pairs
    
    mcq_pairs = []
    batch_size = 10
    for i in range(0, len(nodes), batch_size):
        mcq_pairs += list(
        tqdm(
            ThreadPool().imap(
                lambda x: process_node(*x),
                enumerate(nodes[i:i+batch_size]),
            ),
        f"Generating {engine} QA pairs",
        total=batch_size,
            )
        )
        with open(f"{engine}_MCQs.json", "w") as f:
            json.dump(mcq_pairs, f, indent=4, ensure_ascii=False)
    # flatten
    mcq_pairs = [item for sublist in mcq_pairs for item in sublist]
    return mcq_pairs

# mcq_pairs = generate_mcq_pairs(
#     random_nodes,
#     # nodes,
#     llm,
#     mcq_question_gen_template=mcq_question_gen_template,
#     num_questions_per_chunk=4,
#     delimiter="####"
# )
# for q, a in mcq_pairs:
#     print(f"Q: {q}\nA: {a}\n")

In [25]:
# shuffling the nodes with a fixed seed
import random
random.seed(42)
random.shuffle(nodes)

In [26]:
i = int(len(nodes)*0.5)
gpt4_nodes = nodes[:i]
claude3_nodes = nodes[i:]
all_nodes = [{'gpt-4': gpt4_nodes}, {'claude-3-opus': claude3_nodes}]

In [27]:
len(gpt4_nodes), len(claude3_nodes)

(3044, 3045)

In [ ]:
three_shot_mcq = generate_mcq_pairs(
                                nodes, 
                                mcq_question_gen_template=mcq_question_gen_template,
                                num_questions_per_chunk=2,
                                top_k=3,
                                delimiter="####"
)